In [1]:
#!pip install --upgrade oct2py
import numpy as np
import pandas as pd
from oct2py import octave
import matplotlib.pyplot as plt
import os
from scipy.optimize import root
import pickle
octave.addpath('C:/dynare/6.1/matlab') 
from IPython.display import display
#octave.addpath('/opt/homebrew/opt/dynare/lib/dynare/matlab')

    _pyeval at line 57 column 30

    _pyeval at line 57 column 30



`DyPy` is a class that we create and store in a file called `MyPy.py`. It contains methods for 

* converting data from a Dynare stucture to Numpy arrays.
* analyzing the data. 

See the file for details. 

In [2]:
from MyPy_2 import DyPy

Creates a file called `t_call.m`. This informs Dynare of the format we want for output. It is put here so that adjusments can be made easily; for example the number of periods can be changed here rather than by looking at the `.mod` file. It is included in the `.mod` file via: `@#include "t_call.m"` in `model_1.mod`

* `order`: the order of approximation for the stochastic simulation. order=1 is a fsrst-order approximation (linearization) Higher-order approximations (e.g., order=2 for second-order) capture nonlinear effects in the model, but they require more computational resources.
* `periods`: number of periods (time steps) for which the simulation will be run. periods=1000 means that the model will be simulated for 1000 periods. The default value is 0, which means no simulation will be performed.
* `nograph`: suppresses the graphical output of the simulation results in Dynare since we draw them separatetly. 
* `irf`: the number of periods for which the impulse response functions (IRFs) will be computed and plotted. 

In [3]:
with open('t_call.m', 'w') as file:
    file.write("stoch_simul(nograph, order=1, periods=1000, irf=100);")

 `make_model.ipyn` writes a file called `t_model.m`. `t_model.m`is the `model` block in the Dynare `.mod` file. It can instead be created directlty by commenting out the next `%run` line and coding the model. `make_model.ipyn` simplifies model creation when many of the lines are very similar to each other. Then they can be written in for loops. For the model we are currently working with, there is not much advantage to writing the file via `make_model` but I have done so nonetheless. 
 
 The created file is included in the `.mod` file via `@#include "t_model.m"` in `model_1.mod`. 

In [4]:
%run make_model.ipynb

`make_files.ipynb` writes the other Dynare blocks. See that file for details. However, note that in doing so, it solves for the steady state of the model using `root` from `scipy`. If the results below do not indicate success, the `.mod` file may not run correctly. Also, note that though we solve for the steady state here, we do not give Dynare the steady state but rather use these as starting values for their ss solver. I will later code a verification the steady states are the same. 

In [5]:
%run make_files_1.ipynb

eps,tht,alf,bet,om_0,om_1,om_2,delt,psi_n,psi_k,a_1,a_2,var_a_0,rho_a_0
2,1,0.3,0.9,0.1,0.4,0.5,0.05,1,1,-0.1,1,0.01,0.95


Tells octave where to find the .m file it will run. The file is `call_dynare`. Set to wherever you have this file. 




In [6]:
#a=octave.addpath(os.path.expanduser('C:/Users/blankenw/GitHub/dynare_progs/np_1'))

Makes a call to Octave, which makes a call to Dynare. Then the `.mod` file is run in Dynare. Output from Dynare is displayed as it runs.

In [7]:

# A list of dictionaries representing updates to apply to my_dict:
new_pars = [{'psi_n': 0, 'psi_k': 0}, 
           {'psi_n': 2, 'psi_k': 0}, 
           {'psi_n': 0, 'psi_k': 2}]



# Create an empty dictionary to store your variables
t=[]
with open('mf1.pkl', 'wb') as f:
    pickle.dump((ss_params,model_params,ss_shock), f)


for i, update in enumerate(new_pars):
    model_params.update(update)
    with open('mf1.pkl', 'wb') as f:
        pickle.dump((ss_params,model_params,ss_shock), f)
    %run make_files_2.ipynb
    with open('mf2.pkl', 'rb') as f:
        ss_dict = pickle.load(f)
    # Set up Octave session and run your model
    octave.dynare('model_1.mod')

    # Pull necessary structures
    oo = octave.pull('oo_')
    Mthing = octave.pull('M_')
    instance = DyPy(oo,ss_dict)
    if i==0:
        oo_0=oo
    t.append(instance)


 message: The iteration is not making good progress, as measured by the 
            improvement from the last ten iterations.
 success: False
  status: 5
     fun: [-1.000e-01 -1.000e-01  2.333e-01  1.121e+00  9.997e-02
            4.806e-01  4.822e-01  8.500e-02  1.909e-01  1.108e-01
           -2.683e-01]
       x: [ 3.000e-01  3.000e-01  3.000e-01  3.000e-01  3.000e-01
            3.000e-01  3.000e-01  3.000e-01  3.000e-01  3.000e-01
            1.000e-01]
    nfev: 33
    fjac: [[-5.414e-01  0.000e+00 ...  0.000e+00  0.000e+00]
           [-6.771e-01  0.000e+00 ...  4.130e-01  0.000e+00]
           ...
           [-3.273e-01  2.981e-01 ... -4.629e-02 -1.541e-02]
           [-3.682e-02  3.682e-02 ... -3.919e-09 -1.549e-09]]
       r: [       nan        nan ...        nan        nan]
     qtf: [-3.148e-01  3.299e-01  1.847e-01  2.733e-01  1.408e-01
            4.928e-01  9.359e-01 -4.182e-01  1.237e-01  5.320e-01
           -6.811e-02]


C:\Users\blankenw\AppData\Local\Temp\ipykernel_14528\2185951092.py:8: RuntimeWarning: invalid value encountered in scalar power
  v_0=om_0*c_0**(-1/eps)
C:\Users\blankenw\AppData\Local\Temp\ipykernel_14528\2185951092.py:30: RuntimeWarning: invalid value encountered in scalar power
  eq9=c_0+i-np.exp(a_0)*n_0**(1-alf)*s_0**alf*k**alf


Starting Dynare (version 6.1).
Calling Dynare with arguments: none
Starting preprocessing of the model file ...
Found 10 equation(s).
Evaluating expressions...
Computing static model derivatives (order 1).
Normalizing the static model...
Finding the optimal block decomposition of the static model...
2 block(s) found:
  1 recursive block(s) and 1 simultaneous block(s).
  the largest simultaneous block has 9 equation(s)
                                 and 9 feedback variable(s).
Computing dynamic model derivatives (order 1).
Normalizing the dynamic model...
Finding the optimal block decomposition of the dynamic model...
2 block(s) found:
  1 recursive block(s) and 1 simultaneous block(s).
  the largest simultaneous block has 9 equation(s)
                                 and 9 feedback variable(s).
Preprocessing completed.
Preprocessing time: 0h00m01s.


Residuals of the static equations:
Equation number  1: 1   :       0.425899
Equation number  2: 2   :       2.047328
Equation number  

Oct2PyError: Octave evaluation error:
error: Impossible to find the steady state (the sum of squared residuals of the static equations is 5.5148). Either the model doesn't have a steady state, there are an infinity of steady states, or the guess values are too far from the solution
error: called from:
    print_info at line 33, column 5
    steady at line 143, column 5
    driver at line 498, column 1
    dynare at line 310, column 5

Dynare creates a `structure` which we will need so we `pull` it.

The output from Dynare is stored in a file called `oo_`. This brings `oo_` in as `oo`.

Draw all impulse response functions. 

In [ ]:
def plot_dta(instances, attributes,periods=100):
    plt.figure(figsize=(15, 5))
    for j, attribute in enumerate(attributes):
        # Create a subplot for this attribute
        plt.subplot(1, len(attributes), j+1)
        for i, instance in enumerate(instances):
            # Get the attribute value (time series data)
            attr_value = getattr(instance.ir_level, attribute)
            attr_value = attr_value.reshape(-1, 1)
            length = attr_value.shape[0]
            tm = np.linspace(1, length, length)
            # Add a line to the plot for this time series
            plt.plot(tm[0:periods], attr_value[0:periods], label=labels[i])

        plt.title(attribute)
        plt.legend()

    # Show the figure with all subplots


In [ ]:
labels = ['No adjustment cost', 'Adjustment cost on labor', 'Adjustment cost on capital']

all_attributes =[   #['y_a_0', 'y_a_1', 'y_a_2'],
                    #['y_0_a_0', 'y_0_a_1', 'y_0_a_2'],
                    #['r_a_0', 'r_a_1', 'r_a_2'],
                    #['tot_a_0', 'tot_a_1', 'tot_a_2'],
                    #['sc_a_0', 'sc_a_1', 'sc_a_2'],
                    #['win_a_0', 'win_a_1', 'win_a_2'],
                    #['k_0_a_0', 'k_1_a_0', 'k_2_a_0'],
                    #['n_0_a_0', 'n_1_a_0', 'n_2_a_0'],
                    #['n_0_a_1', 'n_1_a_1', 'n_2_a_1'],
                    #['s_0_a_0', 's_1_a_0', 's_2_a_0'],
                    #['k_a_0', 'i_a_0'],
                    #['k_0_a_1', 'k_1_a_1', 'k_2_a_1'],
                    #['c_0_a_1', 'c_1_a_1', 'c_2_a_1'],
                    #['k_a_1', 'i_a_1'],
                    #['k_0_a_2', 'k_1_a_2', 'k_2_a_2'],
                    #['c_0_a_2', 'c_1_a_2', 'c_2_a_2'],
                    #['k_a_2', 'i_a_2'],
                    #['ns0_a0', 'ns1_a0', 'ns2_a0'],
                    #['ns0_a1', 'ns1_a1', 'ns2_a1'],
                   # ['kn0_a1', 'kn1_a1', 'kn2_a1'],
                    #['ex_0_a_0','ex_1_a_0','ex_2_a_0'],
                    #['exb_0_a_0','exb_1_a_0','exb_2_a_0']
                    
                     ]
for attributes in all_attributes:
    plot_dta(t, attributes,periods=10)
    plt.tight_layout()
    plt.show()                  



In [ ]:
j=1
#n_0=np.insert(t[j].ir_level.n_0_a_0,0,np.exp(t[j].steady_state[0]))
n_1=np.insert(t[j].ir_level.n_1_a_0,0,np.exp(t[j].steady_state[0]))
n_2=np.insert(t[j].ir_level.n_2_a_0,0,np.exp(t[j].steady_state[1]))
n_0=1-n_1-n_2

s_1=np.insert(t[j].ir_level.s_1_a_0,0,np.exp(t[j].steady_state[2]))
s_2=np.insert(t[j].ir_level.s_2_a_0,0,np.exp(t[j].steady_state[3]))
s_0=1-s_1-s_2
k=  np.insert(t[j].ir_level.k_a_0,0,np.exp(t[j].steady_state[4]))
c_0=np.insert(t[j].ir_level.c_0_a_0,0,np.exp(t[j].steady_state[5]))
c_1=np.insert(t[j].ir_level.c_1_a_0,0,np.exp(t[j].steady_state[6]))
c_2=np.insert(t[j].ir_level.c_2_a_0,0,np.exp(t[j].steady_state[7]))
i=  np.insert(t[j].ir_level.i_a_0,0,(t[j].steady_state[8]))
a_0=np.insert(t[j].ir_level.a_0_a_0,0,np.exp(t[j].steady_state[9]))


In [ ]:
i

In [ ]:
s_1+s_2+s_0

In [ ]:
#n_0

In [ ]:
print(max(n_0))
print(max(n_1))
print(max(n_2))
print(min(n_0))
print(min(n_1))
print(min(n_2))

print(max(s_0))
print(max(s_1))
print(max(s_2))
print(min(s_0))
print(min(s_1))
print(min(s_2))
print(min(k))
print(max(k))
print(min(i))
print(max(i))
print(np.mean(c_0))

In [ ]:
om_0=model_params['om_0']
om_1=model_params['om_1']
om_2=model_params['om_2']
eps=model_params['eps']

In [ ]:
p_0=om_0/om_0*(c_0/c_0)**(-1/eps)
y_0=p_0*(c_0+i)


p_1z=p_0*y_0/s_0*s_1/c_1
p_2z=p_0*y_0/s_0*s_2/c_2


p_1=om_1/om_0*(c_1/c_0)**(-1/eps)
p_2=om_2/om_0*(c_2/c_0)**(-1/eps)
y_0=p_0*(c_0+i)
py_0=p_0*y_0
py_1=p_1*c_1
py_2=p_2*c_2


y=py_0+py_1+py_2
r=1
ym=np.roll(y,1)
y_0m=np.roll(y_0,1)
n_0m=np.roll(n_0,1)
n_1m=np.roll(n_1,1)
n_2m=np.roll(n_2,1)
py_0m=np.roll(py_0,1)
py_1m=np.roll(py_1,1)
py_2m=np.roll(py_2,1)

tot=(1/r*y-ym)/ym

sc=(
(1/r*py_0/n_0-py_0m/n_0m)*n_0+
(1/r*py_1/n_1-py_1m/n_1m)*n_1+
(1/r*py_2/n_2-py_2m/n_2m)*n_2
)/ym

wi=(
(n_0-n_0m)*py_0m/n_0m+
(n_1-n_1m)*py_1m/n_1m+
(n_2-n_2m)*py_2m/n_2m
)/ym





In [ ]:
print((p_1/p_2)[0:4])
print((p_1z/p_2z)[0:4])
print((p_1/p_1z)[0:4])
print((p_2/p_2z)[0:4])

In [ ]:
print(n_0[0:4])
print(n_0m[0:4])

In [ ]:
print((s_0)[0:4])
print((n_0)[0:4])

print((s_1)[0:4])
print((n_1)[0:4])
print((s_2)[0:4])


print((n_2)[0:4])

In [ ]:
print((s_0/n_0)[0:4])
print((s_1/n_1)[0:4])
print((s_2/n_2)[0:4])

In [ ]:
print((p_0*y_0/s_0)[0:4])
print((p_1*c_1/s_1)[0:4])
print((p_2*c_2/s_2)[0:4])

In [ ]:
wi_0d=((n_0-n_0m)/n_0m*py_0m)[1:]
wi_1d=((n_1-n_1m)/n_1m*py_1m)[1:]
wi_2d=((n_2-n_2m)/n_2m*py_2m)[1:]

sc_0d=((1/r*py_0/n_0-py_0m/n_0m)*n_0)[1:]
sc_1d=((1/r*py_1/n_1-py_1m/n_1m)*n_1)[1:]
sc_2d=((1/r*py_2/n_2-py_2m/n_2m)*n_2)[1:]

wi=wi[1:]
sc=sc[1:]
tot=tot[1:]


In [ ]:
#wi+sc-tot

In [ ]:
#s_0/n_0-s_1/n_1

In [ ]:
# Create a figure and two subplots in the same row
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4))

# Plot tot, sc, and wi on the first subplot
ax1.plot(tot[0:10], label='tot')
ax1.plot(sc[0:10], label='sc')
ax1.plot(wi[0:10], label='wi')
ax1.legend()
ax1.set_xlabel('Index')
ax1.set_ylabel('Value')
ax1.set_title('Plot of tot, sc, and wi')

# Plot only wi on the second subplot
ax2.plot(wi[0:10], label='wi')
ax2.plot(wi_0d[0:10], label='wi_0d')
ax2.plot(wi_1d[0:10], label='wi_1d')
ax2.plot(wi_2d[0:10], label='wi_2d')
ax2.legend()
ax2.set_xlabel('Index')
ax2.set_ylabel('Value')
ax2.set_title('Plot of wi')

ax3.plot(sc[0:10], label='sc')
ax3.plot(sc_0d[0:10], label='sc_0d')
ax3.plot(sc_1d[0:10], label='sc_1d')
ax3.plot(sc_2d[0:10], label='sc_2d')
ax3.legend()
ax3.set_xlabel('Index')
ax3.set_ylabel('Value')
ax3.set_title('Plot of sc')

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
end

In [ ]:
wi[0:10]

In [ ]:
j=1

n_1=np.exp(t[j].endo_simul[0])
n_2=np.exp(t[j].endo_simul[1])
n_0=1-n_1-n_2

s_1=np.exp(t[j].endo_simul[2])
s_2=np.exp(t[j].endo_simul[3])

s_0=1-s_1-s_2

k=np.exp(t[j].endo_simul[4])
c_0=np.exp(t[j].endo_simul[5])
c_1=np.exp(t[j].endo_simul[6])
c_2=np.exp(t[j].endo_simul[7])
i=(t[j].endo_simul[8])
a_0=np.exp(t[j].endo_simul[9])



#i=np.log(t[j].endo_simul[10])

#i

print(np.mean(n_0[100:]))
print(  )
print(np.mean(n_1[100:]))
print(np.exp(t[j].steady_state[0][0]))
print(  )
print(np.mean(n_2[100:]))
print(np.exp(t[j].steady_state[1][0]))
print(  )
print(np.mean(s_1[100:]))
print(np.exp(t[j].steady_state[2][0]))
print(  )
print(np.mean(s_2[100:]))
print(np.exp(t[j].steady_state[3][0]))
print(  )
print(np.mean(k[100:]))
print(np.exp(t[j].steady_state[4][0]))
print(  )
print(np.mean(c_0[100:]))
print(np.exp(t[j].steady_state[5][0]))
print(  )
print(np.mean(c_1[100:]))
print(np.exp(t[j].steady_state[6][0]))
print(  )
print(np.mean(c_2[100:]))
print(np.exp(t[j].steady_state[7][0]))
print(  )
print(np.mean(i[100:]))
print((t[j].steady_state[8][0]))
print(  )
print(np.mean(a_0[100:]))
print(np.exp(t[j].steady_state[9][0]))






In [ ]:
n_0=n_0[10:]
n_1=n_1[10:]
n_2=n_2[10:]
s_1=s_1[10:]
s_2=s_2[10:]
s_0=s_0[10:]
k=k[10:]
c_0=c_0[10:]
c_1=c_1[10:]
c_2=c_2[10:]
i=i[10:]
a_0=a_0[10:]
a_1=a_1[10:]
a_2=a_2[10:]


In [ ]:
p_0=om_0/om_0*(c_0/c_0)**(-1/eps)
##p_1z=om_1/om_0*(c_1/c_0)**(-1/eps)
#p_2z=om_2/om_0*(c_2/c_0)**(-1/eps)

#
# py_0=p_0*y_0
#py_1=p_1*c_1
#py_2=p_2*c_2
#y_0=p_0*(c_0+i)

y_0=p_0*(c_0+i)


p_1=p_0*y_0/s_0*s_1/c_1
p_2=p_0*y_0/s_0*s_2/c_2
py_0=p_0*y_0
py_1=p_1*c_1
py_2=p_2*c_2

y_0=p_0*(c_0+i)


y=py_0+py_1+py_2
r=1
ym=np.roll(y,1)
y_0m=np.roll(y_0,1)
n_0m=np.roll(n_0,1)
n_1m=np.roll(n_1,1)
n_2m=np.roll(n_2,1)
py_0m=np.roll(py_0,1)
py_1m=np.roll(py_1,1)
py_2m=np.roll(py_2,1)

#ym=ym[1:]
#y_0m=y_0m[1:]
#n_0m=n_0m[1:]
#n_1m=n_1m[1:] 
#n_2m=n_2m[1:]
#py_0m=py_0m[1:]
#py_1m=py_1m[1:]
#py_2m=py_2m[1:]



tot=(1/r*y-ym)/ym

sc=(
(1/r*py_0/n_0-py_0m/n_0m)*n_0+
(1/r*py_1/n_1-py_1m/n_1m)*n_1+
(1/r*py_2/n_2-py_2m/n_2m)*n_2
)/ym

wi=(
(n_0-n_0m)*py_0m/n_0m+
(n_1-n_1m)*py_1m/n_1m+
(n_2-n_2m)*py_2m/n_2m
)/ym


In [ ]:
#p_1/p_1z

In [ ]:
wi

In [ ]:
df = pd.DataFrame({
    'y': y,
    'tot':tot,
    'sc': sc,
    'wi': wi,
    'n_0': n_0,
    'n_1': n_1,
    'n_2': n_2,
    's_0': s_0,
    's_1': s_1,
    's_2': s_2,
    'k': k,
    'c_0': c_0,
    'c_1': c_1,
    'c_2': c_2,
    'i': i,
    'a_0': a_0,
    'py_0': py_0,
    'py_1': py_1,
    'py_2': py_2
    
})

In [ ]:
# Calculate the variance-covariance matrix
df_cov = df.cov()



In [ ]:

# Function to highlight the diagonal
def highlight_diagonal(data):
	attr = 'background-color: yellow'
	# Create a DataFrame with empty strings
	df_styler = pd.DataFrame('', index=data.index, columns=data.columns)
	# Set the diagonal to the desired attribute
	np.fill_diagonal(df_styler.values, attr)
	return df_styler

# Apply the function to the DataFrame
df_cov = df_cov.style.apply(highlight_diagonal, axis=None)

# Display the styled DataFrame
df_cov

In [ ]:
np.std(y)/np.mean(y)

In [ ]:
oo